In [61]:
from abc import abstractmethod
from annoy import AnnoyIndex
import pandas as pd
from google.cloud import aiplatform_v1
from google.cloud import discoveryengine_v1beta as discoveryengine_v1
from google.protobuf.json_format import MessageToDict
import requests
import json

import dotenv
dotenv.load_dotenv()
import google.auth
import google.auth.transport.requests

import re
from google.cloud import storage

import markdownify 

In [87]:


import vertexai
from vertexai.language_models import TextEmbeddingModel
from annoy import AnnoyIndex
import pandas as pd

import requests
import os
from dotenv import load_dotenv
import json
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAI
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
load_dotenv()

vertexai.init(project="vidio-quiz-prod", location="asia-southeast1")
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual")
model = VertexAI(model_name="gemini-pro")

In [155]:

query = "bagaimana mendaftar paket mahasiswa"
# query = "ibukota indonesia"
# query = "1 + 1"
# query = "bayar vidio bisa pakai apa saja"
limit = 10
project_id = '328583281153'
location = 'global'
collection = 'default_collection'
data_store = 'vidio-info_1710991769751'

url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/{location}/collections/{collection}/dataStores/{data_store}/servingConfigs/default_search:search"
auth_req = google.auth.transport.requests.Request()
creds, project = google.auth.default()
creds.refresh(auth_req)


token = creds.token
# data = {"query": query, "pageSize": limit}
data = {
            "query": query,
            "pageSize":10,
            "queryExpansionSpec":{
                "condition":"AUTO"
            },
            "spellCorrectionSpec":{
                "mode":"AUTO"
            },
            "contentSearchSpec":{
                "summarySpec":{
                    "summaryResultCount":5,
                    "ignoreAdversarialQuery":True,
                    "includeCitations":True
                },
                "snippetSpec":{
                    "returnSnippet": True
                },
                "extractiveContentSpec":{
                    "maxExtractiveAnswerCount": 0
                }
            }
        }
headers = {"Authorization": f"Bearer {token}"}
response = requests.post(url, json=data, headers=headers)
response_json = json.loads(response.text)

In [143]:
response_json
# response_json['summary']['summaryText']
# response_json['summary']['summaryWithMetadata']['summary']

{'results': [{'id': '133f11de664b95eb825574e4d3a78c30',
   'document': {'name': 'projects/328583281153/locations/global/collections/default_collection/dataStores/vidio-info_1710991769751/branches/0/documents/133f11de664b95eb825574e4d3a78c30',
    'id': '133f11de664b95eb825574e4d3a78c30',
    'derivedStructData': {'extractive_answers': [{'content': 'Pertandingan dapat disaksikan melalui streaming Vidio! Pekan ini, BRI Liga 1 akan menghadirkan beberapa pertandingan menarik yang layak kamu tunggu seperti Persikabo 1973 vs Persib akan bertemu di Kapten I Wayan Dipta pada Jumat, 15 Maret 2024.'}],
     'link': 'gs://genai_hackathon_2024/data/vidio_info/2024-03-21b/jadwal-lengkap-dan-link-streaming-bri-liga1-2023-2024.html',
     'snippets': [{'snippet_status': 'SUCCESS',
       'snippet': '... <b>1</b>-<b>1</b> Madura United Persija <b>1</b>-<b>1</b> Bali United Senin, 25 September 2023 Borneo FC 1-0 PSM Makassar Pekan 14 Jumat, 29 September 2023 Barito Putera <b>1</b>-<b>1</b> RANS&nbsp;..

In [156]:
response_json['results'][0]['document']['derivedStructData']['snippets'][0]['snippet']

'Bagaimana <b>cara mendaftarkan</b> kampus saya agar mendapatkan <b>paket</b> Vidio Platinum <b>Mahasiswa</b> ? Jika kampus kamu belum terdaftar, kamu bisa <b>mendaftarkan</b> melalui Form&nbsp;...'

In [158]:

response_json['results'][0]['document']['derivedStructData']
# ['extractive_answers'][0]['content']

{'link': 'gs://genai_hackathon_2024/data/faq_html/43000712288.html',
 'title': 'Bagaimana cara mendaftarkan kampus saya agar mendapatkan paket Vidio Platinum Mahasiswa ?',
 'snippets': [{'snippet': 'Bagaimana <b>cara mendaftarkan</b> kampus saya agar mendapatkan <b>paket</b> Vidio Platinum <b>Mahasiswa</b> ? Jika kampus kamu belum terdaftar, kamu bisa <b>mendaftarkan</b> melalui Form&nbsp;...',
   'snippet_status': 'SUCCESS'}]}

In [127]:

# query = "bagaimana mendaftar paket mahasiswa"
query = "berapa harga paket mahasiswa"
# query = "bayar vidio bisa pakai apa saja"
limit = 10
project_id = '328583281153'
location = 'global'
collection = 'default_collection'
data_store = 'vidio-info_1710991769751'

url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/{location}/collections/{collection}/dataStores/{data_store}/servingConfigs/default_search:search"
auth_req = google.auth.transport.requests.Request()
creds, project = google.auth.default()
creds.refresh(auth_req)


token = creds.token
# data = {"query": query, "pageSize": limit}
data = {"query": query,"pageSize":10,"queryExpansionSpec":{"condition":"AUTO"},"spellCorrectionSpec":{"mode":"AUTO"},"contentSearchSpec":{"summarySpec":{"summaryResultCount":5,"ignoreAdversarialQuery":True,"includeCitations":True},"snippetSpec":{"returnSnippet":True},"extractiveContentSpec":{"maxExtractiveAnswerCount":1}}}
headers = {"Authorization": f"Bearer {token}"}
response = requests.post(url, json=data, headers=headers)
response_json = json.loads(response.text)

def get_file(gs_url):
    storage_client = storage.Client("vidio-quiz-prod")

    bucket_name = "genai_hackathon_2024"
    file_path = re.search("gs://genai_hackathon_2024\/(.+)",gs_url).group(1)

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_path)

    return blob.download_as_text()

def to_markdown(html):
    md = markdownify.markdownify(str(html), heading_style="ATX")
    return md

gs_urls = list(map(lambda x: x["document"]["derivedStructData"]["link"], response_json["results"]))
htmls = list(map(get_file, gs_urls))
mds = list(map(to_markdown, htmls))
# result = list(map(lambda x: x["document"]["structData"], response_json["results"]))
# ids = list(map(lambda x: int(x['id']), result))
summary_html = response_json['summary']['summaryWithMetadata']['summary']
summary = markdownify.markdownify(str(summary_html), heading_style="ATX")

information = summary + "\n"
# for html in htmls:
#     information = information + html
for md in mds:
    information = information + md

user_query = query

route_chain = (
    PromptTemplate.from_template(
        """
You are chatbot for Vidio OTT platform. Your task is to answer to user question
Question can is related to vidio. Like about vidio product, and film.
Or question can be unrelated to vidio

Here are the useful information:
<information>
{information} 
</information>

User Question:
<user_query>
{user_query}
</user_query>
"""
    ) | model | StrOutputParser() 
) 

resp = route_chain.invoke({"user_query": user_query, "information": information}) 

Markdown(resp)

Rp 19.000

In [124]:

# response_json['summary']['summaryText']

'Vidio Platinum Mahasiswa adalah <b>paket</b> terbaru yang disediakan khusus untuk Kamu para <b>mahasiswa</b> yang masih aktif, Bisa membeli <b>paket</b> Vidio Platinum 30 hari dengan <b>harga</b> khusus <b>Mahasiswa</b> yaitu Rp. 19.000. *<b>Harga</b> belum termasuk PPN.'

In [115]:
gs_urls

['gs://genai_hackathon_2024/data/faq_html/43000712286.html',
 'gs://genai_hackathon_2024/data/faq_html/43000712283.html',
 'gs://genai_hackathon_2024/data/faq_html/43000711517.html',
 'gs://genai_hackathon_2024/data/faq_html/43000715153.html',
 'gs://genai_hackathon_2024/data/faq_html/43000713388.html',
 'gs://genai_hackathon_2024/data/faq_html/43000712154.html',
 'gs://genai_hackathon_2024/data/faq_html/43000676641.html',
 'gs://genai_hackathon_2024/data/faq_html/43000713394.html',
 'gs://genai_hackathon_2024/data/faq_html/43000708969.html',
 'gs://genai_hackathon_2024/data/faq_html/43000721265.html']

In [117]:
# print(information)
information

'\n\n\nBagaimana cara membeli paket ini ?\n\n\n# Bagaimana cara membeli paket ini ?\n\n\n\nUntuk membeli paket ini, ikuti langkah berikut :\xa0\n\n1. Masuk atau daftar dengan email kampus\n2. Pastikan sedang tidak memiliki paket aktif Platinum di Vidio\n3. Verifikasi email-mu\n4. Beli paketnya via\xa0[Paywall](https://www.vidio.com/plans?fpc=80)\xa0atau halaman\xa0[Platinum Mahasiswa](https://www.vidio.com/pages/234/paket-mahasiswa?layout=false&utm_source=product&utm_medium=faq_page&utm_campaign=platinum_mahasiswa_2023)\n5. Langsung nonton di Vidio!\n\n\n\n\n\n\n\n\nApa itu Vidio Platinum Mahasiswa?\n\n\n# Apa itu Vidio Platinum Mahasiswa?\n\n\n\nPaket terbaru yang disediakan khusus untuk Kamu para mahasiswa yang masih aktif,\xa0\n\nBisa membeli paket Vidio Platinum 30 hari dengan harga khusus Mahasiswa yaitu Rp. 19.000.\n\n\\*Harga belum termasuk PPN \n\n\n\n\n\n\n\n\nBerapa harga paket HSS Berhadiah?\n\n\n# Berapa harga paket HSS Berhadiah?\n\n\n\nHarga paket HSS Berhadiah adalah seb